In [1]:
import numpy as np

In [20]:
# planetary parameters

M_p = 1 # total mass of planet in Earth masses
a = 1 # semi-major axis in AU
Alb = 0.1 # planetary albedo

Ur = 1 # Urey ratio
F_c = 1220e3 / 6371e3 # core fraction in terms of radius
T_0_m = 1800 # initial mantle temperature in K
T_0_c = 2500 # initial core temperature in K
rho_m = 4500 # density of mantle in kg m^-3
rho_c = 11000 # density of core in kg m^-3
Ra_c = 660 # critical Rayleigh number (in Driscoll & Bercovici 2014)
C_pm = 1250 # specific heat capacity mantle
alpha_m = 2e-5 # thermal expansivity of silicate mantle
k_m = 4 # thermal conductivity of silicate mantle
X_K = 2 # initial abundance of K in wt %
X_U = 1e-2# initial abundane of U in wt %
X_Th = 1e-2 # initial abundance of Th in wt %

# model parameters
tau_f = 4.5e9 # final time in years
dtau = 10e6 # time-step in years

kappa_m = thermal_diffusivity(k_m, rho_m, C_pm)
M_E = 5.972e24 # earth mass in kg
years2sec = 31557600
params = dict(M_p=M_p*M_E, a=a, Alb=Alb, F_c=F_c, T_0_m=T_0_m, T_0_c=T_0_c, rho_lid=rho_lid, rho_m=rho_m,
              rho_c=rho_c, Ra_c=Ra_c, C_pm=C_pm, alpha_m=alpha_m, k_m=k_m, X_K=X_K, X_U=X_U, X_Th=X_Th,
              kappa_m=kappa_m, Ur=Ur, tau_f=tau_f*years2sec, dtau=dtau*years2sec)

In [6]:
def secular_cooling(dtau, M, C_v, q_r, H, R_outer, R_inner=0):
    """Secular cooling equation. Ignores adiabatic heating from contraction, external energy transfers, 
    work by atmospheric pressure. Assumes spatially uniform heat flux and heat production. 
    Might assume this is 0 (Urey ratio of unity)
    
    Parameters
    ----------
    dtau : float
        Time step
    M : float
        Mass of reservoir in kg
    C_v : float 
        Specific heat by volume  
    q_r : float 
        Heat flux in W m**-2 
    H : float
        Volumetric internal heat production in W m**-3
    R_outer : float
        Outer radius of reservoir in m
    R_inner : float (optional)
        Inner radius of reservoir in m. Defaults to 0
        

    Returns
    -------
    dT : float
        Surface temperature change over time step in K
    """
    
    SA = 4*np.pi*R**2 # outer surface area of reservoir
    V = 4/3*np.pi*(R_outer**3 - R_inner**3) # volume of reservoir
    dT = (-q_r*SA + H*V) / (M*C_v) * dtau
    return dT

def surf_temp(a, Alb, q_m=0, S=1361):
    """Calculate surface temperature assuming no greenhouse effect

    Parameters
    ----------
    a : float
        Semi-major axis in AU
    q_m : float
        Heat flux from mantle
    S : float (optional)
        Solar constant at 1 AU and 4.5 Gyr in W m^-2. Defaults to solar value    
    Al : float 
        Albedo. Defaults to 0.1 (arbitrarily assumed)
        

    Returns
    -------
    T_surf : float
        Surface temperature in K
    """
    
    sb = 5.670374419e-8 # Stefen-Boltzmann constant in W m^−2 K^−4
    # TODO: solar luminosity increases with age, e.g. ~30% over 2 Gyr
    T_surf = ( (((1 - Alb)*S / (4*a**2)) - q_m) / sb )**0.25
    return T_surf
    
def rad_flux(tau, K_0, X_K, U_0_235, U_0_238, X_U, Th_0, X_Th):
    """Calculate radiative heating from K-U-Th decay
    
    Parameters
    ----------
    tau : float
        Time elapsed
    X_K : float
        Abundance of K accreted in wt. %
    K_0 : float
        Ratio of 40-K to 39-K at tau = 0
        
    Returns
    -------
    h : float
        Radiative heat production in W kg^-1
    """
    
    # Half-lives in years from Dye (2012) in Treatise on Geophys
    t_40K_half = 1.26e9 
    t_235U_half = 7.04e8 
    t_238U_half = 4.46e9
    t_232Th_half = 1.4e10
    
    # Heating rates of radioisotopes at tau=0 in W kg^-1 from Dye (2012) in Treatise on Geophys
    h_40K_0 = 28.47e-6
    h_235U_0 = 568.47e-6
    h_238U_0 = 95.13e-6
    h_232Th_0 = 26.3e-6
        
    h_40K = h_40K_0 * K_0 * X_K * np.exp(np.log(2)*(tau/t_40K_half))
    h_235U = h_235U_0 * U_0_235 * X_U * np.exp(np.log(2)*(tau/t_235U_half))
    h_238U = h_238U_0 * U_0_238 * X_U * np.exp(np.log(2)*(tau/t_238U_half))
    h_232Th = h_232Th_0 * Th_0 * X_Th * np.exp(np.log(2)*(tau/t_232Th_half))
    h = h_40K + h_235U + h_238U + h_232Th
    return h

def surf_flux(A, k, T_s, T_m, d_lid):
    """Heat conduction through upper boundary layer which limits convective cooling
    
    Parameters
    ----------
    A : float
        Surface area in m^2 [problem: sphere??]
    k : float
        Thermal conductivity of upper mantle
    T_s : float
        Surface temperature in K
    T_m : float
        Mantle temperature in K
    d_lid : float
        Crust thickness in m
    
    Returns
    ------
    q : float
        Heat flux in W m^-2  [OR W m^-1???]
    """
    return A*k*(T_m - T_s)/d_lid
    
def CMB_flux(Ur): 
    """Calculate heat flux across core-mantle boundary, assuming it is governed by Urey ratio
    
    Parameters
    ----------
    Ur : Urey ratio
    """

    return q_CMB # W m**-2

def lid_thickness(T_s, T_m, Ra_c, kappa_m, alpha_m, g):
    """Thickness of uppper mantle thermal boundary layer 
    
    Parameters
    ----------
    T_s : float
        Surface temperature
    Ra_c : float
        Critical Rayleigh number
    alpha_m : float 
        Thermal expansivity in
    kappa : float 
        Thermal diffusivity in
    nu_m : float
        Mantle viscosity in
    g : float
        Acceleration due to gravity in m s^-2
    
        
    Returns
    -------
    d : float
        Lid thickness in m
    """
    
    # Set Rayleigh number at critical value
    d = (Ra_c * nu_m*kappa_m/(alpha_m*g*(T_m - T_s)))**(1/3)
    return d

def planet_radius(M_p, F_c, rho_m, rho_c):
    """Calculate radius of planet given total mass
    
    Parameters
    ----------
    M_p : float
        Mass of planet
    rho_m : float
        Mantle density
    F_c : float
        Core fraction in terms of radius
    rho_c : float
        Core density
    """
    
    R_p = ( 3*M_p/(4*np.pi) / (rho_m - F_c**3*rho_m + F_c**3*rho_c) )**(1/3)
    return R_p

def gravity(M_p, R_p):
    """Calculate acceleration due to gravity in m s^-2"""
    return 6.674e-11*M_p/R_p**2

def thermal_diffusivity(k, rho, C_p):
    """
    Calculate thermal diffusivity
    
    Parameters
    ----------
    k : Thermal conductivity
    C_p : Specific heat capacity in J K^-1 kg^-1
    rho : density in kg m^-3
    """
    return k/(rho*C_p)

In [ ]:
def start(params)
    for ii in range(0, dtau, tau_f):
        step(**params)

    
def step(tau, **params):
    """Advance one time step"""
    
    R_c = R_p*F_c 
    
    # 0. Calculate planet radius, surface area
    R_p = planet_radius(M_p, F_c, rho_m, rho_c, d_lid)
    SA_p = 4*np.pi*R_p**2
    g = gravity(M_p, R_p)
    
    # 1. Evolve surface temperature (set by astrophysics)
    T_s = T_surf(a, Alb)
    
    # 2. Calculate lid thickness and new mantle volume
    d_lid = lid_thickness(T_s, T_m, Ra_c, kappa_m, alpha_m, g)
    V_m = 4/3*np.pi*((R_p - d_lid)**3 - (F_c*R_p)**3)
    
    # 3. Calculate internal heating (crust and mantle) --- only radiogenic for now
    H_rad = rad_flux(tau, K_0, X_K, U_0_235, U_0_238, X_U, Th_0, X_Th)
    H_lid = H_rad
    H_m = H_rad
    
    # 4. Calculate mantle fluxes
    q_m_in = CMB_flux(T_m) # CMB
    q_m_out = surf_flux(SA_p, k_m, T_s, T_m, d_lid)  
    
    # Calculate new mantle temperature
    delta_T_m = (-q_m_out*SA_p + H_m*rho_m*V_m + q_m_in*SA_c) / (rho_m*C_pm*V_m) * dtau
    
    # Calculate new core temperature
    V_c = 4/3*np.pi*R_c**3
    delta_T_c = -q_m_in*R_c / (rho_c*C_pm*V_c) * dtau
    
    # TODO: use Ur=1 to complete budget for core


In [ ]:
def Ra(nu, alpha, rho, g, T_m, T_s, d, kappa, ):
    """Calculate Rayleigh number of convecting material

    Parameters
    ----------
    nu : float
        Viscosity
    alpha : float 
        Thermal expansivity
    rho : float
        Density
    kappa : float 
        Thermal diffusivity 
    d : float
        Thickness of convecting layer
    """
    
    g = 9.81 # Acceleration due to gravity
    
    return alpha*rho*g*(T_m - T_s)*d**3 / (kappa*nu)

def viscosity(T, nu_0, A_v):
    """Calculate Rayleigh number of convecting material

    Parameters
    ----------
    T : float 
        Temperature in K
    nu_0 : float
        Reference viscosity
    A_v : float
        Activation energy
    """
    
    R_g = # gas constant
    nu = nu_0 * np.exp(A_v)
    
    # isoviscous for now
    return nu_0



In [ ]:
def rad_heating(C_p, tau, dtau):
    """Calculate change in temperature in K due to radiogenic heating

    Parameters
    ----------
    C_p : float 
        Heat capacity of the silicate material   
    dtau : float 
        Time step 
    """
    h = rad_flux(tau)
    return h/C_p * dtau